In [13]:
import sys
sys.path.append("./drive_interfaces/carla/carla_client/")
from carla.client import make_carla_client
from carla.settings import CarlaSettings
from carla.client import VehicleControl

In [83]:
with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
    print(f.read())

; the server related info will not be in effect when launching through python
[CARLA/Server]
UseNetworking=true
SynchronousMode = true
SendNonPlayerAgentsInfo= true

[CARLA/QualitySettings]
; Quality level of the graphics, a lower level makes the simulation run
; considerably faster. Available: Low or Epic.
QualityLevel=Epic

[CARLA/LevelSettings]
NumberOfVehicles=0
NumberOfPedestrians=0
WeatherId=1
; Path of the vehicle class to be used for the player. Leave empty for default.
; Paths follow the pattern "/Game/Blueprints/Vehicles/Mustang/Mustang.Mustang_C"
PlayerVehicle=

[CARLA/Sensor]
Sensors=CameraLeft,CameraMiddle,CameraRight,DepthLeft,DepthMiddle,DepthRight,SegLeft,SegMiddle,SegRight
ImageSizeX=800
ImageSizeY=600
; 70
FOV=100
PositionX=2.50
PositionY=0
; 0.5
PositionZ=1.00
; 8
RotationPitch=0
RotationRoll=0
RotationYaw=0
SensorType=CAMERA


[CARLA/Sensor/CameraLeft]
PostProcessing=SceneFinal
RotationYaw=-30.0
[CARLA/Sensor/CameraMiddle]
PostProcessing=SceneFinal
RotationYaw=0
[CA

In [52]:
import random
from carla.sensor import Camera, Lidar
settings = CarlaSettings()
settings.set(
    SynchronousMode=True,
    SendNonPlayerAgentsInfo=True,
    NumberOfVehicles=20,
    NumberOfPedestrians=40,
    WeatherId=random.choice([1, 3, 7, 8, 14]),
    QualityLevel="Low")
settings.randomize_seeds()

# Now we want to add a couple of cameras to the player vehicle.
# We will collect the images produced by these cameras every
# frame.

# The default camera captures RGB images of the scene.
camera0 = Camera('CameraRGB')
# Set image resolution in pixels.
camera0.set_image_size(800, 600)
# Set its position relative to the car in meters.
camera0.set_position(0.30, 0, 1.30)
settings.add_sensor(camera0)

# Let's add another camera producing ground-truth depth.
camera1 = Camera('CameraDepth', PostProcessing='Depth')
camera1.set_image_size(800, 600)
camera1.set_position(0.30, 0, 1.30)
settings.add_sensor(camera1)

print(str(settings))

[CARLA/Server]
SynchronousMode=True
SendNonPlayerAgentsInfo=True

[CARLA/QualitySettings]
QualityLevel=Low

[CARLA/LevelSettings]
NumberOfVehicles=20
NumberOfPedestrians=40
WeatherId=8
SeedVehicles=48887
SeedPedestrians=13506
DisableTwoWheeledVehicles=False

[CARLA/Sensor]
Sensors=CameraRGB,CameraDepth

[CARLA/Sensor/CameraRGB]
FOV=90.0
ImageSizeX=800
ImageSizeY=600
PositionX=0.3
PositionY=0
PositionZ=1.3
PostProcessing=SceneFinal
RotationPitch=0.0
RotationRoll=0.0
RotationYaw=0.0
SensorName=CameraRGB
SensorType=CAMERA

[CARLA/Sensor/CameraDepth]
FOV=90.0
ImageSizeX=800
ImageSizeY=600
PositionX=0.3
PositionY=0
PositionZ=1.3
PostProcessing=Depth
RotationPitch=0.0
RotationRoll=0.0
RotationYaw=0.0
SensorName=CameraDepth
SensorType=CAMERA




In [103]:
from carla.client import CarlaClient
client = CarlaClient("127.0.0.1", 2000)
client.connect()
with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
    #client.load_settings(settings)
    client.load_settings(f.read())
client.start_episode(0)
measurements, sensor_data = client.read_data()
client.send_control(VehicleControl())

client.disconnect()

In [84]:
with make_carla_client("127.0.0.1", 2000) as client:
    with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
        #client.load_settings(settings)
        client.load_settings(f.read())
    client.start_episode(0)
    measurements, sensor_data = client.read_data()
    client.send_control(VehicleControl())

platform_timestamp: 2147483648
game_timestamp: 800
player_measurements {
  transform {
    location {
      x: 271.040008545
      y: 129.489990234
      z: 39.3734817505
    }
    orientation {
      x: -1.0
      y: 4.05311584473e-06
    }
    rotation {
      yaw: 179.999755859
    }
  }
  acceleration {
    x: -4.56150184291e-09
    y: 1.84882948659e-14
  }
  forward_speed: 1.82460069276e-09
  autopilot_control {
    steer: 3.26974054587e-06
    throttle: 1.0
  }
  bounding_box {
    transform {
      location {
        z: 0.659999966621
      }
      orientation {
        x: 1.0
      }
      rotation {
      }
    }
    extent {
      x: 2.33999991417
      y: 0.939999997616
      z: 0.649999976158
    }
  }
}
non_player_agents {
  id: 2118317720
  traffic_light {
    transform {
      location {
        x: 94.9899749756
        y: 70.4099960327
        z: 38.2132415771
      }
      orientation {
        x: 1.0
        y: 1.7310541125e-06
      }
      rotation {
        yaw: 9.

In [89]:
measurements.player_measurements

transform {
  location {
    x: 271.040008545
    y: 129.489990234
    z: 39.2184524536
  }
  orientation {
    x: -1.0
    y: 4.05311584473e-06
  }
  rotation {
    yaw: 179.999755859
  }
}
acceleration {
  x: -4.55694282309e-09
  y: 1.84698176892e-14
}
forward_speed: 3.64737773317e-09
autopilot_control {
  steer: 3.26974054587e-06
  throttle: 1.0
  reverse: true
}
bounding_box {
  transform {
    location {
      z: 0.659999966621
    }
    orientation {
      x: 1.0
    }
    rotation {
    }
  }
  extent {
    x: 2.33999991417
    y: 0.939999997616
    z: 0.649999976158
  }
}

In [96]:
sensor_data.keys()

[u'SegRight',
 u'CameraLeft',
 u'DepthLeft',
 u'CameraMiddle',
 u'SegLeft',
 u'CameraRight',
 u'DepthMiddle',
 u'SegMiddle',
 u'DepthRight']

In [101]:
image_converter.depth_to_array(sensor_data['DepthLeft']).shape


(600, 800)

In [91]:
a=sensor_data['DepthLeft']

In [92]:
from carla import image_converter

In [94]:
b=image_converter.depth_to_array(a)
b.shape

(600, 800)

In [95]:
b

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.0023784 , 0.0023784 , 0.0023784 , ..., 0.0023784 , 0.0023784 ,
        0.0023784 ],
       [0.00237048, 0.00237048, 0.00237048, ..., 0.00237048, 0.00237048,
        0.00237048],
       [0.00236255, 0.00236255, 0.00236255, ..., 0.00236255, 0.00236255,
        0.00236255]])

In [16]:
measurements

platform_timestamp: 2147483648
game_timestamp: 1200
player_measurements {
  transform {
    location {
      x: 271.040008545
      y: 129.489990234
      z: 39.2184524536
    }
    orientation {
      x: -1.0
      y: 4.05311584473e-06
    }
    rotation {
      yaw: 179.999755859
    }
  }
  acceleration {
    x: -4.55694282309e-09
    y: 1.84698176892e-14
  }
  forward_speed: 3.64737773317e-09
  autopilot_control {
    steer: 3.26974054587e-06
    throttle: 1.0
    reverse: true
  }
  bounding_box {
    transform {
      location {
        z: 0.659999966621
      }
      orientation {
        x: 1.0
      }
      rotation {
      }
    }
    extent {
      x: 2.33999991417
      y: 0.939999997616
      z: 0.649999976158
    }
  }
}
non_player_agents {
  id: 3922395471
  traffic_light {
    transform {
      location {
        x: 94.9899749756
        y: 70.4099960327
        z: 38.2132415771
      }
      orientation {
        x: 1.0
        y: 1.7310541125e-06
      }
      rotatio

AttributeError: 'GeneratorContextManager' object has no attribute 'load_settings'

In [ ]:
measure, sensor = 